In [1]:
data_root = 'drumData/'
sr = 48000 # this is the samplerate initially used to load the samples
total_limit = 100 #None # set this to 100 to export 100 samples
length_limit = sr/4 # set this to sr/4 to only export 250ms of audio per sample
downsampling_factor = 4
drumSprites = {}
limit = None

In [5]:
%matplotlib inline
from matplotlib import pyplot as plt
from os.path import join
from utils import ffmpeg_save_audio
import numpy as np
from multiprocessing import Pool
import pickle
import time as timeMod
import scipy.spatial
import scipy.spatial.distance as dist 
import itertools
from utils import *
from datetime import datetime
from time import mktime
from time import time
from sklearn.decomposition import PCA
#%time samples = np.load(join(data_root, 'samples.npy'))

# Load samples

In [3]:
drumNames = pickle.load(open(data_root+'drumNames.pickle'))
drumLengths = pickle.load(open(data_root+'drumLengths.pickle'))
drumFingerPrints = {}
drumSamples = {}
for d in drumNames:
    %time drumSamples[d] = np.load(join(data_root, d+'_samples.npy'))
colors = ['#000000', '#ff0000', '#00ff00', '#0000ff', '#ffff00', '#ff00ff', '#00ffff']
initial_dims = [30]
perplexities = [30]

CPU times: user 464 µs, sys: 227 ms, total: 227 ms
Wall time: 355 ms
CPU times: user 636 µs, sys: 18.6 ms, total: 19.2 ms
Wall time: 26.5 ms
CPU times: user 513 µs, sys: 104 ms, total: 104 ms
Wall time: 151 ms
CPU times: user 1.12 ms, sys: 58.5 ms, total: 59.7 ms
Wall time: 80.1 ms
CPU times: user 528 µs, sys: 7.49 ms, total: 8.02 ms
Wall time: 11.1 ms
CPU times: user 473 µs, sys: 9.96 ms, total: 10.4 ms
Wall time: 15.3 ms
CPU times: user 546 µs, sys: 30.4 ms, total: 31 ms
Wall time: 57.3 ms


# Compute sprites

In [4]:
# take a sample, crop it, and down sample it
def job(samp, length_limit, downsampling_factor):
    shortSamp = samp[:length_limit]
    if len(shortSamp) < length_limit:
        shortSamp = np.pad(shortSamp, (0, length_limit-len(shortSamp)), 'constant', constant_values=0)
    downShortSamp = np.interp(np.arange(0, length_limit, downsampling_factor), np.arange(0, length_limit), shortSamp)
    return downShortSamp

def calculateSprites(length_limit, downsampling_factor):
    _drumSprites = {}
    for d in drumNames:
        y = drumSamples[d]
        sprites = map(lambda samp: job(samp, length_limit, downsampling_factor), drumSamples[d][:limit])
        sprites = np.asarray(sprites).astype(np.float32)
        _drumSprites[d] = sprites
        print "generated spritesheet print for", d, sprites.shape
    return _drumSprites

drumSprites = calculateSprites(length_limit, downsampling_factor)
data = np.concatenate([drumSprites[d] for d in drumNames])


generated spritesheet print for kick (5158, 3000)
generated spritesheet print for tom (422, 3000)
generated spritesheet print for snare (2546, 3000)
generated spritesheet print for clap (1324, 3000)
generated spritesheet print for hi.hat (159, 3000)
generated spritesheet print for ride (228, 3000)
generated spritesheet print for crash (723, 3000)


# define TSNE calculation

In [6]:
X_2d_inspect = None

def save_data(data, fn):
    np.savetxt(fn, data, fmt='%.5f', delimiter='\t')

def savePlotsAndData(newData, data_root, prefix, colorMap, dataDir, plotDir, initial_dims=30, perplexity=30):    
    figsize = (16,16)
    pointsize = 30
    
    struct = timeMod.localtime(time())
    dt = datetime.fromtimestamp(mktime(struct))
    
    save_data(newData, join(data_root, dataDir+'/{}.{}.{}.2d - {}.tsv'.format(prefix, initial_dims, perplexity, dt)))
    
    plt.figure(figsize=figsize)
    plt.scatter(newData[:,0], newData[:,1], c=colorMap, s=pointsize)
    plt.tight_layout()
    plt.savefig(join(data_root, plotDir+'/{}.{}.{}_2D - {}.png'.format(prefix, initial_dims, perplexity, dt)))
    plt.close()
    
def tsne(data, data_root, prefix, colorMap, initial_dims=30, perplexity=30):
    mkdir_p(data_root + 'tsne')
    mkdir_p(data_root + 'plot')
    
    print initial_dims, perplexity, type(data), data.shape, data.dtype
    X_2d = list(bh_tsne(data, initial_dims=initial_dims, perplexity=perplexity, no_dims=2, verbose=True))
    X_2d = normalize(np.array(X_2d))
    
    savePlotsAndData(X_2d, data_root, prefix, colorMap, 'tsne', 'plot', initial_dims, perplexity)
    
    return X_2d

def pca(data, data_root, prefix, colorMap):
    mkdir_p(data_root + 'pca')
    mkdir_p(data_root + 'pcaPlot')
    
    pcaInstance = PCA(n_components=2)  
    X_2d = pcaInstance.fit_transform(data)
    
    savePlotsAndData(X_2d, data_root, prefix, colorMap, 'pca', 'pcaPlot')
    
    return X_2d

    
def concatColors(segmentList, colorList):
    multiples = []
    #print segmentList, colorList
    for i in range(len(segmentList)):
        multiples.append([colorList[i]]*segmentList[i])
    return list(itertools.chain(*multiples))

### Do dimensionality reduction

In [7]:
data = data.astype(np.float64)
colorMap = concatColors(drumLengths, colors)
useTsne = True
prefix = 'sprites'
def job(params):
    start = time()
    tsne2d = tsne(data, data_root, prefix, colorMap, initial_dims=params[0], perplexity=params[1])
    pca2d = pca(data, data_root, prefix, colorMap)
    print 'initial_dims={}, perplexity={}, {} seconds'.format(params[0], params[1], time() - start)
    return [tsne2d, pca2d]
params = list(itertools.product(initial_dims, perplexities))
pool = Pool()
dimReducedArrays = pool.map(job, params)
newData = dimReducedArrays[0]

30 30 <type 'numpy.ndarray'> (10560, 3000) float64
initial_dims=30, perplexity=30, 167.770179987 seconds


### Common setup 

In [8]:
# create point -> class hashmap
def getClassesPerSample(data):
    drumClasses = {}
    classIndex = 0
    for i in range(len(data)):
        if sum(drumLengths[0:classIndex+1]) <= i:
            classIndex += 1
        drumClasses[tuple(data[i])] = drumNames[classIndex]
    return drumClasses

drumClasses = getClassesPerSample(data)

numNeighbors = 10

## Arguments
## data -    an array of length n where array[i] is the fraction of neighbors of point i 
#            had the same class as point i.
# numPerClass - Assumes that points from the same class are contiguous and in
#            the same order as drumNames. Should use drumLengths for this parameter
# calcFunc - The summary statistic you want to calcuate per class (mean, medain, etc)
def calculateFuncPerClass(data, numPerClass, calcFunc):
    segments = [0]+numPerClass
    for i in range(1, len(segments)):
        segments[i] = segments[i] + segments[i-1]
    valuesPerClass = []
    for i in range(len(segments)-1):
        valuesPerClass.append(calcFunc(data[segments[i]:segments[i+1]]))
    return valuesPerClass

### pairwise distance implementation

In [9]:
#using https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.spatial.distance.cdist.html
#and argpartition from - https://stackoverflow.com/questions/6910641/how-to-get-indices-of-n-maximum-values-in-a-numpy-array

## Arguments - pointData is m x n np array, with n points of m dimensions
# pointD numNeighbors is the number of nearest neighbors for which to compare classes
#
## returns - an array of length n where array[i] is the fraction of neighbors of point i 
#            had the same class as point i
def calculateKNNClassAccuracy(pointData, numNeighbors, printTimes=False):
    startTime = timeMod.time()
    pairwiseDist = dist.cdist(pointData, pointData)
    if printTimes:
        print "pairwise distances calculated", timeMod.time() - startTime
    kPartition = np.argpartition(pairwiseDist, -numNeighbors)
    if printTimes:
        print "partitions calculated", timeMod.time() - startTime
    fracSameNeighborClasses = np.zeros((len(data)))
    
    for i in range(len(pairwiseDist)):
        neighborIndexes = kPartition[i][-numNeighbors:]
        neighbors = [data[ind] for ind in neighborIndexes]
        
        sampleClass = drumClasses[tuple(data[i])]
        neighborClasses = [drumClasses[tuple(neighbor)] for neighbor in neighbors]
        numSameNeighborClasses = len(filter(lambda c : c == sampleClass, neighborClasses))
        fracSameNeighborClasses[i] = numSameNeighborClasses * 1.0 / numNeighbors
    
    if printTimes:
        print "knn classes calculated", timeMod.time() - startTime
        
    return fracSameNeighborClasses

classAccuracies_LD_TSNE = calculateKNNClassAccuracy(newData[0], numNeighbors, True)
classAccuracies_LD_PCA = calculateKNNClassAccuracy(newData[0], numNeighbors, True)
classAccuracies_HD = calculateKNNClassAccuracy(data, numNeighbors, True)

pairwise distances calculated 1.07847809792
partitions calculated 2.96025514603
knn classes calculated 29.0214810371
pairwise distances calculated 0.878396987915
partitions calculated 2.18531298637
knn classes calculated 27.8770859241
pairwise distances calculated 369.047304153
partitions calculated 370.24468708
knn classes calculated 395.923006058


In [12]:
print "mean for highDim, tsne, pca"
print round(np.mean(classAccuracies_HD), 3), round(np.mean(classAccuracies_LD_TSNE), 3), round(np.mean(classAccuracies_LD_PCA), 3)
print

print "std for highDim, tsne, pca"
print round(np.std(classAccuracies_HD), 3), round(np.std(classAccuracies_LD_TSNE), 3), round(np.std(classAccuracies_LD_PCA), 3)
print

print "mean per class for highDim"
print [round(n, 3) for n in calculateFuncPerClass(classAccuracies_HD, drumLengths, np.mean)]
print 

print "std per class for highDim"
print [round(n, 3) for n in calculateFuncPerClass(classAccuracies_LD_TSNE, drumLengths, np.std)]
print

print "mean per class for tsne"
print [round(n, 3) for n in calculateFuncPerClass(classAccuracies_LD_TSNE, drumLengths, np.mean)]
print 

print "std per class for tsne"
print [round(n, 3) for n in calculateFuncPerClass(classAccuracies_HD, drumLengths, np.std)]
print

print "mean per class for pca"
print [round(n, 3) for n in calculateFuncPerClass(classAccuracies_LD_PCA, drumLengths, np.mean)]
print 

print "std per class for pca"
print [round(n, 3) for n in calculateFuncPerClass(classAccuracies_LD_PCA, drumLengths, np.std)]
print

mean for highDim, tsne, pca
0.469 0.231 0.231

std for highDim, tsne, pca
0.475 0.401 0.401

mean per class for highDim
[0.953, 0.095, 0.0, 0.0, 0.0, 0.0, 0.0]

std per class for highDim
[0.465, 0.017, 0.027, 0.029, 0.0, 0.0, 0.051]

mean per class for tsne
[0.47, 0.002, 0.002, 0.003, 0.0, 0.0, 0.004]

std per class for tsne
[0.052, 0.023, 0.0, 0.0, 0.0, 0.0, 0.0]

mean per class for pca
[0.47, 0.002, 0.002, 0.003, 0.0, 0.0, 0.004]

std per class for pca
[0.465, 0.017, 0.027, 0.029, 0.0, 0.0, 0.051]

